In [1]:
import torch
import torch.utils.data
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments, AutoConfig, T5Model, T5Tokenizer
import random

In [2]:
# print torch version
print(torch.__version__)


2.0.0+cu117


In [3]:
seed = 42
rand = random.Random(seed)
# generate a random text with random length
def random_text():
    length = rand.randint(5, 200)
    return ''.join([rand.choice('abcdefghijklmnopqrstuvwxyz') for _ in range(length)])

In [14]:
# Load the ByT5 model
model_name = "google/byt5-small"
# model = T5Model.from_pretrained(model_name)
# also we want a raw model without pretraining
# Load the ByT5 tokenizer
# tokenizer = T5Tokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


loading configuration file config.json from cache at /home/janpro/.cache/huggingface/hub/models--google--byt5-small/snapshots/68377bdc18a2ffec8a0533fef03b1c513a4dd49d/config.json
Model config T5Config {
  "_name_or_path": "google/byt5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3584,
  "d_kv": 64,
  "d_model": 1472,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 4,
  "num_heads": 6,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "ByT5Tokenizer",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 384
}

loading weights fi

In [5]:

# config = AutoConfig.from_pretrained(model_name)
# model_raw = T5Model(config)
# model = torch.compile(model)
# model_raw = torch.compile(model_raw)



In [15]:


# Prepare your data for training, 
# the task is to copy the input to the output

# create a dataset where input_ids and labels are the same
class CopyDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, size):
        self.input = [random_text() for _ in range(size)]
        self.tokenizer = tokenizer
    def __getitem__(self, idx):
        return {"input_ids": tokenizer(self.input[idx])["input_ids"], "labels": tokenizer(self.input[idx])["input_ids"]}
    def __len__(self):
        return len(self.input)


In [16]:

train_dataset = CopyDataset(tokenizer, size=1000) 
dev_dataset = CopyDataset(tokenizer, size=100)


# put it in a dataset
# train dev test split
# train_dataset, dev_dataset, test_dataset = torch.utils.data.random_split(dataset, [8000, 1000, 1000], generator=torch.Generator().manual_seed(seed))


In [18]:

# Define your training parameters
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="byt5-copytuned",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    push_to_hub=False,



    logging_dir="./logs",
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Train the model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.train()
#ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/janpro/enigma-transformed/enigmavenv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1875
  Number of trainable parameters = 299637760


  0%|          | 0/1875 [00:00<?, ?it/s]